In [1]:


import cv2
import numpy as np
import json
import os
from pathlib import Path
import matplotlib.pyplot as plt


class CallusAnalyzer:
    """
    Sistema de análisis automático de callos vegetales en placas de Petri
    Utiliza OpenCV para segmentación y medición de parámetros
    """
    
    def __init__(self, output_dir="resultados"):
        self.output_dir = output_dir
        Path(output_dir).mkdir(exist_ok=True)
        
    def load_image(self, image_path):
        """Carga imagen desde archivo"""
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"No se pudo cargar la imagen: {image_path}")
        return img
    
    def preprocess_image(self, img):
        """
        Pre-procesamiento de la imagen:
        1. Conversión a escala de grises
        2. Filtrado gaussiano para reducir ruido
        3. Mejora de contraste con CLAHE
        """
        # Convertir a escala de grises
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Aplicar filtro gaussiano para suavizar
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # Mejorar contraste con CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        enhanced = clahe.apply(blurred)
        
        return enhanced
    
    def segment_callus_rgb(self, img):
        """
        Segmentación usando análisis de color RGB
        Los callos vegetales suelen tener tonos amarillos/verdosos
        """
        # Convertir a HSV para mejor segmentación por color
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
        # Definir rangos de color para callos vegetales (tonos amarillo-verde)
        lower_yellow = np.array([15, 30, 30])
        upper_yellow = np.array([85, 255, 255])
        
        # Crear máscara de color
        mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        
        return mask
    
    def segment_callus_threshold(self, preprocessed):
        """
        Segmentación usando umbralización adaptativa
        """
        # Umbralización de Otsu
        _, binary = cv2.threshold(preprocessed, 0, 255, 
                                  cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        return binary
    
    def morphological_operations(self, binary_mask):
        """
        Operaciones morfológicas para limpiar la máscara:
        1. Opening: eliminar ruido pequeño
        2. Closing: rellenar huecos
        """
        # Crear kernels morfológicos
        kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
        
        # Opening para eliminar ruido
        opened = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel_open)
        
        # Closing para rellenar huecos
        closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel_close)
        
        return closed
    
    def analyze_blobs(self, binary_mask, img_original, min_area=500):
        """
        Análisis de blobs (callos) usando análisis de componentes conectadas
        Calcula área, centroide, y otras características
        """
        # Encontrar contornos
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, 
                                       cv2.CHAIN_APPROX_SIMPLE)
        
        callus_data = []
        img_result = img_original.copy()
        
        callus_count = 0
        for i, contour in enumerate(contours):
            # Calcular área
            area = cv2.contourArea(contour)
            
            # Filtrar callos pequeños (ruido)
            if area < min_area:
                continue
            
            callus_count += 1
            
            # Calcular parámetros del callo
            M = cv2.moments(contour)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
            else:
                cx, cy = 0, 0
            
            # Calcular perímetro
            perimeter = cv2.arcLength(contour, True)
            
            # Calcular circularidad (4π*área/perímetro²)
            if perimeter > 0:
                circularity = 4 * np.pi * area / (perimeter ** 2)
            else:
                circularity = 0
            
            # Calcular rectángulo delimitador
            x, y, w, h = cv2.boundingRect(contour)
            
            # Guardar datos del callo
            callus_data.append({
                'id': callus_count,
                'area_pixels': float(area),
                'perimetro_pixels': float(perimeter),
                'centroide_x': cx,
                'centroide_y': cy,
                'ancho_pixels': w,
                'alto_pixels': h,
                'circularidad': float(circularity)
            })
            
            # Dibujar contornos y etiquetas en la imagen
            cv2.drawContours(img_result, [contour], -1, (0, 255, 0), 2)
            cv2.circle(img_result, (cx, cy), 5, (255, 0, 0), -1)
            cv2.putText(img_result, f"#{callus_count}", (cx-20, cy-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            cv2.putText(img_result, f"{area:.0f}px²", (cx-30, cy+20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return callus_data, img_result, callus_count
    
    def process_image(self, image_path, visualize=True):
        """
        Pipeline completo de procesamiento de una imagen
        """
        # 1. Cargar imagen
        img = self.load_image(image_path)
        img_name = os.path.basename(image_path)
        
        print(f"\nProcesando: {img_name}")
        print(f"Dimensiones: {img.shape[1]}x{img.shape[0]} píxeles")
        
        # 2. Pre-procesar
        preprocessed = self.preprocess_image(img)
        
        # 3. Segmentar (combinar métodos)
        mask_rgb = self.segment_callus_rgb(img)
        mask_thresh = self.segment_callus_threshold(preprocessed)
        
        # Combinar máscaras
        combined_mask = cv2.bitwise_or(mask_rgb, mask_thresh)
        
        # 4. Operaciones morfológicas
        clean_mask = self.morphological_operations(combined_mask)
        
        # 5. Analizar blobs
        callus_data, img_result, count = self.analyze_blobs(clean_mask, img)
        
        print(f"Callos detectados: {count}")
        
        # Visualización
        if visualize:
            self.visualize_results(img, preprocessed, clean_mask, 
                                  img_result, img_name)
        
        return {
            'imagen': img_name,
            'num_callos': count,
            'callos': callus_data
        }
    
    def visualize_results(self, img_original, preprocessed, mask, 
                         img_result, img_name):
        """Visualizar resultados del procesamiento"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # Imagen original
        axes[0, 0].imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))
        axes[0, 0].set_title('Imagen Original')
        axes[0, 0].axis('off')
        
        # Pre-procesamiento
        axes[0, 1].imshow(preprocessed, cmap='gray')
        axes[0, 1].set_title('Pre-procesamiento')
        axes[0, 1].axis('off')
        
        # Máscara binaria
        axes[1, 0].imshow(mask, cmap='gray')
        axes[1, 0].set_title('Segmentación')
        axes[1, 0].axis('off')
        
        # Resultado final
        axes[1, 1].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
        axes[1, 1].set_title('Detección de Callos')
        axes[1, 1].axis('off')
        
        plt.tight_layout()
        output_path = os.path.join(self.output_dir, 
                                   f"analisis_{img_name}")
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        plt.close()
        
    def process_batch(self, image_folder):
        """
        Procesar un lote de imágenes
        """
        # Buscar todas las imágenes PNG en la carpeta
        image_paths = list(Path(image_folder).glob("*.png"))
        
        if not image_paths:
            print(f"No se encontraron imágenes PNG en {image_folder}")
            return
        
        print(f"Encontradas {len(image_paths)} imágenes")
        
        all_results = []
        
        for img_path in image_paths:
            try:
                result = self.process_image(str(img_path), visualize=True)
                all_results.append(result)
            except Exception as e:
                print(f"Error procesando {img_path}: {e}")
        
        # Guardar resultados
        self.save_results(all_results)
        self.generate_summary(all_results)
        
        return all_results
    
    def save_results(self, results):
        """Guardar resultados en archivo JSON"""
        output_path = os.path.join(self.output_dir, "resultados_analisis.json")
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        print(f"\n✓ Resultados guardados en: {output_path}")
    
    def generate_summary(self, results):
        """Generar resumen en archivo de texto"""
        output_path = os.path.join(self.output_dir, "resumen_analisis.txt")
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("RESUMEN DE ANÁLISIS DE CALLOS VEGETALES\n")
            f.write("=" * 80 + "\n\n")
            
            total_callos = sum(r['num_callos'] for r in results)
            f.write(f"Total de imágenes procesadas: {len(results)}\n")
            f.write(f"Total de callos detectados: {total_callos}\n")
            f.write(f"Promedio de callos por imagen: {total_callos/len(results):.2f}\n\n")
            
            f.write("-" * 80 + "\n")
            f.write("DETALLE POR IMAGEN\n")
            f.write("-" * 80 + "\n\n")
            
            for result in results:
                f.write(f"\nImagen: {result['imagen']}\n")
                f.write(f"Número de callos: {result['num_callos']}\n")
                
                if result['callos']:
                    f.write("\nDetalle de cada callo:\n")
                    for callo in result['callos']:
                        f.write(f"  - Callo #{callo['id']}:\n")
                        f.write(f"    * Área: {callo['area_pixels']:.2f} píxeles²\n")
                        f.write(f"    * Perímetro: {callo['perimetro_pixels']:.2f} píxeles\n")
                        f.write(f"    * Circularidad: {callo['circularidad']:.3f}\n")
                        f.write(f"    * Dimensiones: {callo['ancho_pixels']}x{callo['alto_pixels']} px\n")
                
                f.write("\n" + "-" * 80 + "\n")
        
        print(f"✓ Resumen guardado en: {output_path}")



In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
def print_acquisition_system():
    """
    Imprime especificaciones del sistema de adquisición recomendado
    """
    print("\n" + "=" * 80)
    print("SISTEMA DE ADQUISICIÓN DE IMÁGENES RECOMENDADO")
    print("=" * 80 + "\n")
    
    print("1. CÁMARA:")
    print("   - Tipo: Cámara digital científica o DSLR")
    print("   - Resolución mínima: 5 MP (2592x1944 píxeles)")
    print("   - Sensor: CMOS o CCD")
    print("   - Profundidad de color: 24 bits (RGB)")
    print("   - Ejemplo: Basler acA2440-20gc, Canon EOS 2000D\n")
    
    print("2. ÓPTICA:")
    print("   - Objetivo: Macro o teleobjetivo corto (50-100mm)")
    print("   - Distancia focal fija para consistencia")
    print("   - Apertura: f/8 - f/11 (buena profundidad de campo)")
    print("   - Campo de visión: ~15x20 cm (cubrir placa Petri completa)\n")
    
    print("3. ILUMINACIÓN:")
    print("   - Tipo: LED anular o caja de luz difusa")
    print("   - Temperatura de color: 5000-6500K (luz día)")
    print("   - Intensidad uniforme para evitar sombras")
    print("   - Difusores para luz suave y homogénea\n")
    
    print("4. SOPORTE Y POSICIONAMIENTO:")
    print("   - Soporte vertical estable (copy stand)")
    print("   - Altura fija: ~40-50 cm sobre la muestra")
    print("   - Plataforma horizontal nivelada")
    print("   - Fondo de color neutro (blanco o gris)\n")
    
    print("5. CALIBRACIÓN:")
    print("   - Incluir regla o patrón de referencia en cada toma")
    print("   - Distancia objeto-cámara constante")
    print("   - Resolución espacial: ~0.1-0.2 mm/píxel")
    print("   - Realizar calibración geométrica periódica\n")
    
    print("6. PARÁMETROS DE CAPTURA:")
    print("   - Formato: PNG o TIFF (sin compresión con pérdida)")
    print("   - Balance de blancos: automático o manual fijo")
    print("   - ISO: bajo (100-400) para minimizar ruido")
    print("   - Velocidad obturación: según iluminación\n")
    
    print("=" * 80 + "\n")


In [6]:
if __name__ == "__main__":
    # Mostrar sistema de adquisición recomendado
    print_acquisition_system()
    
    # Crear analizador
    analyzer = CallusAnalyzer(output_dir="resultados_callos")
    
    # Procesar imágenes
    # OPCIÓN 1: Procesar un lote de imágenes en una carpeta
    print("\nIniciando análisis de imágenes...")
    print("=" * 80)
    
    # Cambiar 'imagenes_callos' por la ruta de tu carpeta
    folder_path = "C:/Users/angel/Desktop/labelme_annotation/labelme_annotation"
    
    if os.path.exists(folder_path):
        results = analyzer.process_batch(folder_path)


SISTEMA DE ADQUISICIÓN DE IMÁGENES RECOMENDADO

1. CÁMARA:
   - Tipo: Cámara digital científica o DSLR
   - Resolución mínima: 5 MP (2592x1944 píxeles)
   - Sensor: CMOS o CCD
   - Profundidad de color: 24 bits (RGB)
   - Ejemplo: Basler acA2440-20gc, Canon EOS 2000D

2. ÓPTICA:
   - Objetivo: Macro o teleobjetivo corto (50-100mm)
   - Distancia focal fija para consistencia
   - Apertura: f/8 - f/11 (buena profundidad de campo)
   - Campo de visión: ~15x20 cm (cubrir placa Petri completa)

3. ILUMINACIÓN:
   - Tipo: LED anular o caja de luz difusa
   - Temperatura de color: 5000-6500K (luz día)
   - Intensidad uniforme para evitar sombras
   - Difusores para luz suave y homogénea

4. SOPORTE Y POSICIONAMIENTO:
   - Soporte vertical estable (copy stand)
   - Altura fija: ~40-50 cm sobre la muestra
   - Plataforma horizontal nivelada
   - Fondo de color neutro (blanco o gris)

5. CALIBRACIÓN:
   - Incluir regla o patrón de referencia en cada toma
   - Distancia objeto-cámara constante
 